In [1]:
from datetime import datetime
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import pandas as pd
import yahoo_fin.stock_info as si
from utils.utils_bigquery import *
from datetime import *
import yfinance as yf
import json

In [2]:
key_path = key_path
project = project_id
dataset = 'bronze'
table = 'bronze_ticker_info'
table_conca = f'{project}.{dataset}.{table}'

In [3]:
# Conectamos con Bigquery
bigquery = BigQueryUtils(key_path)

In [4]:
unique_tickers = get_unique_tickers()

In [5]:
import yfinance as yf
import pandas as pd

data = []

for i, row in unique_tickers.iterrows():
    ticker = row['ticker']
    
    # Obtener datos históricos
    ticker_data = yf.Ticker(ticker).info
    
    # Convertir el diccionario en una fila de DataFrame
    data_row = pd.Series(ticker_data)
    
    # Agregar el símbolo como una columna en los datos históricos
    data_row['ticker'] = ticker
    
    # Añadir los datos a la lista
    data.append(data_row)

# Concatenar todas las filas y convertir en un DataFrame
df = pd.DataFrame(data)
df.head()


,address1,city,state,zip,country,phone,website,industry,industryKey,industryDisp,...,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio,ticker,fax,address2,industrySymbol
0,One Apple Park Way,Cupertino,CA,95014,United States,408 996 1010,https://www.apple.com,Consumer Electronics,consumer-electronics,Consumer Electronics,...,0.049,0.45962,0.34175,0.29556,USD,2.1854,AAPL,NaN,NaN,NaN
1,One Amgen Center Drive,Thousand Oaks,CA,91320-1799,United States,805 447 1000,https://www.amgen.com,Drug Manufacturers - General,drug-manufacturers-general,Drug Manufacturers - General,...,0.201,0.63412,0.37283,0.22759,USD,2.3802,AMGN,805 447 1010,NaN,NaN
2,410 Terry Avenue North,Seattle,WA,98109-5210,United States,206 266 1000,https://www.aboutamazon.com,Internet Retail,internet-retail,Internet Retail,...,0.101,0.48043,0.17217,0.09915,USD,2.1533,AMZN,NaN,NaN,NaN
3,200 Vesey Street,New York,NY,10285,United States,212 640 2000,https://www.americanexpress.com,Credit Services,credit-services,Credit Services,...,0.087,0.55833,NaN,0.21082,USD,2.4064,AXP,NaN,NaN,NaN
4,929 Long Bridge Drive,Arlington,VA,22202,United States,703-465-3500,https://www.boeing.com,Aerospace & Defense,aerospace-defense,Aerospace & Defense,...,-0.146,0.10463,0.01797,-0.05698,USD,NaN,BA,NaN,NaN,NaN


In [6]:
# Identificar columnas de tipo 'object'
object_columns = df.select_dtypes(include=['object']).columns

# Convertir las columnas de tipo 'object' a listas
for col in object_columns:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, (list, dict)) else x)

In [7]:
# Guardamos los datos en bigquery
bigquery.save_dataframe(df, project, dataset, table, if_exists='replace', schema=None)

100%|██████████| 1/1 [00:00<?, ?it/s]
